In [2]:
import pandas as pd
from sklearn import preprocessing

import src

In [26]:
pd.set_option("display.max_colwidth", None)

# Load data

In [12]:
df = pd.read_parquet(src.PATH / "data/raw/sentences.parquet.gzip")
preds = pd.read_parquet(src.PATH / "data/interim/sentence_predictions.parquet.gzip")
gruendl = pd.read_parquet(src.PATH / "data/interim/gruendl.parquet.gzip")
gruendl["gruendl"] = gruendl["gruendl"].astype(int)

df = pd.merge(df, preds, on="sample_id")
df = pd.merge(df, gruendl, on="sample_id")

In [13]:
thresh = {"elite": 0.415961, "pplcentr": 0.295400, "left": 0.429109, "right": 0.302714}

In [14]:
elite_ids = ("Anti-Elitism", [421345, 288335, 1033085])
centr_ids = ("People-Centrism", [1087337, 575681, 231293])
pop_ids = ("Populism", [1064158, 1091032, 1195880])

In [30]:
results = []
for cat, ids in [elite_ids, centr_ids, pop_ids]:
    sub_df = df.loc[
        df.sample_id.isin(ids), ["text", "elite", "gruendl", "pplcentr", "left", "right"]
    ].copy()
    sub_df["category"] = cat
    results.append(sub_df)

sample_df = pd.concat(results)
sample_df = sample_df.round(2)
sample_df["id"] = range(1, len(sample_df) + 1)
sample_df = sample_df[
    ["category", "id", "text", "gruendl", "elite", "pplcentr", "left", "right"]
].rename(
    {
        "category": "Dimension",
        "id": "ID",
        "text": "Text",
        "gruendl": "Gründl (2022)",
        "pplcentr": "centr",
    },
    axis=1,
)
sample_df["Text"] = sample_df["Text"].apply(lambda x: f"\parbox[t]{{7cm}}{{{x}}}")

In [38]:
sample_df

,Dimension,ID,Text,Gründl (2022),elite,centr,left,right
288334,Anti-Elitism,1,\parbox[t]{7cm}{Die Deregulierung der Finanzmärkte hat doch erst die Dollarzeichen in die Augen der Spekulanten gezaubert.},1,0.96,0.02,0.97,0.02
421344,Anti-Elitism,2,\parbox[t]{7cm}{Die Opposition befindet sich intellektuell bereits in der Sommerpause.},0,0.89,0.00,0.01,0.02
1033084,Anti-Elitism,3,"\parbox[t]{7cm}{Für uns ist dies ein richtiger und notwendiger Schritt, er ist erforderlich, und wir sehen noch Nachholbedarf: 42,9 Milliarden Euro in 2019 und 60 Milliarden Euro in 2023.}",0,0.01,0.01,0.00,0.00
231292,People-Centrism,4,"\parbox[t]{7cm}{Ein Syrer mit 4 Frauen und 23 Kindern kostet den deutschen Steuerzahler schon jetzt im Jahr 400 000 Euro Alimentierung, ohne je einen einzigen Cent eingezahlt zu haben.}",0,0.06,0.90,0.19,0.71
575680,People-Centrism,5,"\parbox[t]{7cm}{Wir fordern für diese Menschen nicht irgendwelche sozialpolitischen Maßnahmen, sondern wir fordern: Schluss mit der dauerhaften gesellschaftlichen Ausgrenzung!}",0,0.00,0.68,0.72,0.01
1087336,People-Centrism,6,\parbox[t]{7cm}{Wir machen konkrete Politik für die ganz konkreten Probleme der Menschen.},0,0.00,0.90,0.07,0.02
1064157,Populism,7,\parbox[t]{7cm}{Für Mieterinnen und Mieter kommt diese Tatenlosigkeit der Regierung wirklich teuer.},0,0.99,0.13,0.08,0.01
1091031,Populism,8,"\parbox[t]{7cm}{Das ist Klassenkampf von oben, das ist Klassenkampf im Interesse von Vermögenden und Besitzenden gegen die Mehrheit der Steuerzahlerinnen und Steuerzahler auf dieser Erde.}",1,0.88,0.35,0.98,0.02
1195879,Populism,9,"\parbox[t]{7cm}{Sie sind jene, die den Gender Pay Gap vergrößern, indem Sie die armen deutschen Rentnerinnen mit Füßen treten, während Sie den Asylbewerbern, die hauptsächlich jung und männlich sind, das Geld in den Rachen werfen.}",0,0.96,0.68,0.10,0.98


In [39]:
sample_df.to_latex(src.PATH / "results/tables/selected_examples.tex", index=False, escape=False)

/tmp/ipykernel_58688/2533246234.py:1: FutureWarning: In future versions `DataFrame.to_latex` is expected to utilise the base implementation of `Styler.to_latex` for formatting and rendering. The arguments signature may therefore change. It is recommended instead to use `DataFrame.style.to_latex` which also contains additional functionality.
  sample_df.to_latex(src.PATH / "results/tables/selected_examples.tex", index=False, escape=False)
